In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from gensim.models import FastText
from gensim.models.fasttext import load_facebook_vectors
import fasttext
import io

import pandas as pd
from collections import Counter
from torch.utils.data import DataLoader

In [2]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, context_size, embedding_dim, hidden_dim):
        super(CBOW, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.context_size = context_size
        
        # nn.Embedding is a matrix of learnable weights, one row per word
        self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim)
        # embedding vectors to be projected into a hidden layer, context size * 2 because skip gram is bef-y-aft
        self.linear1 = nn.Linear(self.context_size * 2 * self.embedding_dim, hidden_dim)
        # finally fully connect and softmax to output is the target y in bef-y-aft
        self.linear2 = nn.Linear(hidden_dim, self.vocab_size) # fully connect hidden layer to (log) softmax

    def forward(self, inputs):
        out = self.embeddings(inputs)
        out = out.view(1, -1) # multiple words in the skipgram.
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.log_softmax(out, dim=1) # log probabilities
        return(out)

# function to create the context tensor
def context_vector(context, word_to_ix):
    return torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

def target_index(target, word_to_ix):
    return(torch.tensor([word_to_ix[target]], dtype=torch.long))

def train(model, loss_function, optimiser, skipgrams, word_to_ix):
    
    device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    losses = []
    for epoch in range(10):
        total_loss = 0
        for batch, (context, target) in enumerate(skipgrams):

            # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
            # into integer indices and wrap them in tensors)
            context_idxs = context_vector(context, word_to_ix)
            
            # Step 2. Recall that torch *accumulates* gradients. Before passing in a
            # new instance, you need to zero out the gradients from the old
            # instance
            model.zero_grad()

            # Step 3. Run the forward pass, getting log probabilities over next
            # words
            log_probs = model(context_idxs.to(device))

            # Step 4. Compute your loss function. (Again, Torch wants the target
            # word wrapped in a tensor)
            target = target_index(target, word_to_ix)
            loss = loss_function(log_probs, target.to(device))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimiser.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
            if batch % 10000 == 0:
                print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
        losses.append(total_loss)
    return(losses)

# similarity of two words
def test(model, word_1, word_2, word_to_ix):
    # test word similarity
    word_1_vec = model.embeddings.weight[word_to_ix[word_1]]
    word_2_vec = model.embeddings.weight[word_to_ix[word_2]]
    
    dot_product = word_1_vec.dot(word_2_vec)
    norm = torch.norm(word_1_vec) * torch.norm(word_2_vec)
    word_similarity = (dot_product / norm)
    print(f"Similarity between {word_1} & {word_2} : {word_similarity}")
    

def create_skipgrams(text_stream, context_size):
    # count unique
    word_counts = Counter(text_stream)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    vocab_size = len(vocab)

    word_to_ix = {word: i for i, word in enumerate(vocab)}
    
    skipgrams = []
    for i in range(context_size, len(text_stream) - context_size):
        context = (
            [text_stream[i - j - 1] for j in range(context_size)],
            [text_stream[i + j + 1] for j in range(context_size)]
        )
        context[0].reverse()
        context = context[0] + context[1]
        target = text_stream[i]
        skipgrams.append((context, target))
    return(vocab_size, word_to_ix, skipgrams)    

In [3]:
context_size = 2  # before and after. consequently needs doubling when counting tensor sizes based on it.

raw_text1 = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

train_df = pd.read_csv('data/reddit-cleanjokes.csv')
jokes = train_df['Joke'].str.cat(sep=' ')
raw_jokes = jokes.split(' ')

# need to do pre-processing (remove puncs, lower)
# set up a padding idx so that the opening and closing words can be included.
# how can stopwords be handled

vocab_size, word_to_ix, skipgrams = create_skipgrams(raw_jokes, 2)

In [7]:
embedding_dim = 128
hidden_dim = 32
learning_rate = 0.05
model = CBOW(vocab_size, context_size, embedding_dim, hidden_dim)
optimiser = optim.SGD(model.parameters(), lr=learning_rate)
loss_function = nn.NLLLoss() # this is the loss function for multi-class with a log_softmax final layer

losses = train(model, loss_function, optimiser, skipgrams, word_to_ix)

{'epoch': 0, 'batch': 0, 'loss': 8.593474388122559}
{'epoch': 0, 'batch': 10000, 'loss': 5.983700752258301}
{'epoch': 0, 'batch': 20000, 'loss': 3.7963109016418457}
{'epoch': 1, 'batch': 0, 'loss': 3.15470552444458}
{'epoch': 1, 'batch': 10000, 'loss': 4.485655784606934}
{'epoch': 1, 'batch': 20000, 'loss': 0.0182971004396677}
{'epoch': 2, 'batch': 0, 'loss': 3.153709650039673}
{'epoch': 2, 'batch': 10000, 'loss': 4.524255275726318}
{'epoch': 2, 'batch': 20000, 'loss': 3.706634521484375}
{'epoch': 3, 'batch': 0, 'loss': 3.1088781356811523}
{'epoch': 3, 'batch': 10000, 'loss': 4.352497100830078}
{'epoch': 3, 'batch': 20000, 'loss': 3.7364792823791504}
{'epoch': 4, 'batch': 0, 'loss': 3.135406017303467}
{'epoch': 4, 'batch': 10000, 'loss': 4.372165679931641}
{'epoch': 4, 'batch': 20000, 'loss': 3.6914873123168945}
{'epoch': 5, 'batch': 0, 'loss': 3.0685043334960938}
{'epoch': 5, 'batch': 10000, 'loss': 4.469064712524414}
{'epoch': 5, 'batch': 20000, 'loss': 3.723297357559204}
{'epoch': 6

In [8]:
print("training loss captured")
print(losses)  # The loss decreased every iteration over the training data!
print()
# take a look at a couple of them
print("a sample of in_grams and targets")
print(skipgrams[:2])
print()
# converting to context vector
print("a context vector")
print(context_vector(skipgrams[0][0], word_to_ix))  # example
print()
# To get the embedding of a particular word
print("an embedding")
print(model.embeddings.weight[word_to_ix["to"]]) # a word all sets will have in common
print()
print("similarities")
test(model, "cat", "dog", word_to_ix)
test(model, "cat", "bartender", word_to_ix)

training loss captured
[189333.9221344018, 171245.48541558493, 168841.02656311047, 167670.80985538344, 167353.86152805228, 167756.6030868171, 168678.98116172053, 169098.32764983742, 169428.90714260947, 169738.395093128]

a sample of in_grams and targets
[(['What', 'did', 'bartender', 'say'], 'the'), (['did', 'the', 'say', 'to'], 'bartender')]

a context vector
tensor([  2,   8, 248,  20])

an embedding
tensor([ 1.2659, -0.8588,  0.8405, -0.6843,  1.3552, -0.4044, -1.0121, -0.3106,
         2.0615,  0.5692,  0.0199, -0.3875,  1.1145,  0.3165,  0.7183, -0.9846,
        -1.2619, -0.7782,  1.4091,  0.6360, -0.8570,  1.4968,  0.6001, -0.9534,
        -0.2191,  1.3916,  0.9696,  0.9899,  0.3182, -0.4750,  0.6714,  1.0217,
        -1.4073,  0.9290, -1.2338, -1.3542, -1.0391,  0.9319,  1.0203, -0.7688,
        -0.9297,  0.5050, -1.1625, -1.5972,  0.3227, -1.1030, -1.2720, -1.0534,
        -0.7546,  0.8573, -0.0682,  0.8937, -1.7024, -1.1347, -1.2212,  0.4738,
        -0.0481, -1.4430, -0.1769,

In [36]:
# now with fasttext
# needs a file
with open("data/jokes_string.txt","w") as f:
    f.write(jokes)

model = fasttext.train_unsupervised("data/jokes_string.txt", model="cbow",
                                    dim=32, # window=5,
                                    minn=2, maxn=4,
                                    epoch=10, lr=0.05)

#  print([model.get_word_vector(x) for x in ["asparagus", "pidgey", "yellow"]])
model.get_nearest_neighbors('cat')

[(0.9996886253356934, 'count'),
 (0.9996806979179382, 'get'),
 (0.9996606707572937, 'college?'),
 (0.9996600151062012, 'without'),
 (0.9996529221534729, 'through'),
 (0.9996510744094849, 'car?'),
 (0.9996461868286133, "couldn't"),
 (0.9996421337127686, 'left'),
 (0.9996353983879089, 'legs?'),
 (0.9996336698532104, 'that')]

In [67]:
# with FastText curtesy of gensim
model = FastText(vector_size=64, window=3, min_count=1)  # instantiate
sentences = [j.split(" ") for j in train_df['Joke']]
model.build_vocab(sentences)
model.train(sentences, total_examples=train_df['Joke'].count(), epochs=10)  # train

(186541, 239140)

In [69]:
model.wv["bartender"]

array([-0.0830541 , -0.20610413, -0.02786405,  0.21481003, -0.10866459,
       -0.558164  , -0.6200512 ,  0.19819567, -0.315694  , -0.45674908,
       -0.24560012,  0.5245908 ,  0.03480747,  0.38249454, -0.14317559,
       -0.02465119, -0.01064481,  0.00741511, -0.735859  ,  0.37666684,
       -0.7486268 , -0.0088387 ,  0.47247148,  0.30037135,  0.05788086,
        0.3069808 , -0.4259284 ,  0.20305124,  0.48619732, -0.5880369 ,
       -0.10926346, -0.143354  ,  0.2501288 , -0.24491423, -0.3084543 ,
       -0.27975678, -0.3055602 , -0.47814012,  0.2035015 , -0.45709568,
       -0.3310865 , -0.24251764,  0.73030865,  0.24830015, -0.41156146,
       -0.3562771 ,  0.32073572, -0.20648572, -0.06289098,  0.33641177,
        0.12991379,  0.11348884,  0.39474407,  0.45570472,  0.06986182,
        0.14759685, -0.27532235, -0.28736055,  0.02858935, -0.34413934,
        0.12848435, -0.37820527, -0.08179902, -0.13455968], dtype=float32)

In [70]:
# (path, encoding='utf-8')
help("load_facebook_vectors")

No Python documentation found for 'load_facebook_vectors'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

